# Prisoners Dilemma

In [1]:
# !pip install gym[classic_control]
# !pip install keras-rl2
# !pip install protobuf==3.20.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.2
    Uninstalling pygame-2.5.2:
      Successfully uninstalled pygame-2.5.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 909.1 kB/s eta 0:00:00


In [1]:
import tensorflow as tf
from tensorflow import keras as k
import numpy as np
import gym
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from keras import __version__
tf.keras.__version__ = __version__

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


from gym import Env
from gym.spaces import Discrete, Box, MultiBinary
import numpy as np
import random

from collections import deque

In [3]:
!pip list

Package                       Version
----------------------------- ---------
absl-py                       1.4.0
aiohttp                       3.8.5
aiosignal                     1.3.1
altair                        5.3.0
asttokens                     2.2.1
astunparse                    1.6.3
async-timeout                 4.0.2
attrs                         23.1.0
awkward                       2.6.6
awkward-cpp                   35
backcall                      0.2.0
beautifulsoup4                4.12.2
blinker                       1.8.2
cachetools                    5.3.2
calmjs.parse                  1.3.1
cernopendata-client           0.3.0
certifi                       2023.5.7
charset-normalizer            3.1.0
click                         8.1.6
cloudpickle                   3.0.0
colorama                      0.4.6
comm                          0.1.2
contourpy                     1.0.7
cramjam                       2.8.3
cycler                        0.11.0
debugpy            


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Setting Up the Environment

In [4]:
def tit_for_tat(round,opp_move):
  if round == 200:
    return 1
  else:
    return opp_move

In [5]:
def cooperate():
  return 1

def defect():
  return 0

def rand():
  return random.randint(0,1)

In [6]:
def tit_for_two_tat(round, opp_move1, opp_move2):
  if round >= 199 or opp_move1 == 1 or opp_move2 == 1:
    return 1
  else:
    return 0

In [7]:
def grudger(defected):
  if defected:
    return 0
  else:
    return 1

In [8]:
def pavlov(round, last_move,opp_move):
  if round == 200:
    return 1
  elif last_move == opp_move or (last_move == 0 and opp_move == 1):
    return last_move
  elif last_move == 1 and opp_move == 0:
    return 0

In [9]:
def soft_grudger(last_defect):
  if last_defect < 4:
    return 0
  else:
    return 1

In [10]:
def fortress(round, opp_move1, opp_move2):
  if round == 200 or opp_move1 == 0 or opp_move2 == 0:
    return 0
  else:
    return 1

In [11]:
def alternator(round,last_move):
  if round == 200 or last_move == 0:
    return 1
  else:
    return 0

In [12]:
def sneaky(round,opp_move):
  dice=random.randint(0,9)
  if round == 200:
    return 1
  elif dice == 0:
    return 0;
  else:
    return opp_move

In [13]:
def custom(round):
  if round%4 == 0:
    return 0
  else:
    return 1

In [14]:
class PrisonEnv(Env):
    def __init__(self):
        self.action_space = Discrete(2)
        self.observation_space = MultiBinary(10)
        self.round = 200
        self.state = np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,-1])
        self.opp_strategy = random.randint(0,10)
        self.opp_move1=1
        self.opp_move2=1
        self.last_move1=self.state[0]
        self.last_move2=self.state[1]
        self.opp_defected=False
        self.user_defected=False
        self.last_defect= np.Infinity
        self.opp_last_defect= np.Infinity
        self.move=1
        self.opp_move=1

        self.opp_score=0

    def step(self, action):

        self.opp_move2=self.opp_move1
        self.opp_move1=self.opp_move
        self.last_move2=self.state[1]
        self.last_move1=self.state[0]

        self.move=action



        if self.opp_strategy == 0:
          opp_move=tit_for_tat(self.round,self.last_move1)
        elif self.opp_strategy == 1:
          opp_move=cooperate()
        elif self.opp_strategy == 2:
          opp_move=sneaky(self.round, self.last_move1)
        elif self.opp_strategy == 3:
          opp_move=rand()
        elif self.opp_strategy == 4:
          opp_move=tit_for_two_tat(self.round,self.last_move1, self.last_move2)
        elif self.opp_strategy == 5:
          opp_move=grudger(self.user_defected)
        elif self.opp_strategy == 6:
          opp_move=pavlov(self.round,self.opp_move1, self.last_move1)
        elif self.opp_strategy == 7:
          opp_move=soft_grudger(self.last_defect)
        elif self.opp_strategy == 8:
          opp_move=fortress(self.round, self.last_move1, self.last_move2)
        elif self.opp_strategy == 9:
          opp_move=alternator(self.round,self.opp_move1)
        elif self.opp_strategy == 10:
          opp_move = custom(self.round)


        temp = np.array(list(self.state))

        for i in range(1,10):
          self.state[i] = temp[i-1]
        self.state[0] = self.move


        if opp_move == 0:
          self.opp_defected=True
          self.opp_last_defect=0
        else:
          self.opp_last_defect += 1

        if self.move == 0:
          self.user_defected=True
          self.last_defect=0
        else:
          self.last_defect += 1

        if self.move == 1 and opp_move == 1:
          reward = 3
          self.opp_score += 3
        elif self.move == 1 and opp_move == 0:
          reward = 0
          self.opp_score += 5
        elif self.move == 0 and opp_move == 1:
          reward = 5
          self.opp_score += 0
        elif self.move == 0 and opp_move == 0:
          reward = 1
          self.opp_score += 1

        self.round -= 1

        if self.round <= 0:
            done = True

        else:
            done = False

        info = {'opp_score':self.opp_score}



        return self.state, reward, done, info


    def reset(self):
        self.round = 200
        self.state = np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,-1])
        self.opp_strategy = random.randint(0,10)
        self.opp_move1=1
        self.opp_move2=1
        self.last_move1=self.state[0]
        self.last_move2=self.state[1]
        self.opp_defected=False
        self.user_defected=False
        self.last_defect= np.Infinity
        self.opp_last_defect= np.Infinity
        self.move=1
        self.opp_move=1

        self.opp_score = 0

        return self.state

In [15]:
env = PrisonEnv()
env.state

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [16]:
data = env.state.reshape(-1, 10)
data[0]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

# Test on Random Values

In [17]:
env = PrisonEnv()
episodes = 100
done = False
score = 0
wins=0
for episode in range(1,episodes+1):
  state=env.reset()
  done=False
  score=0
  while not done:
      action = env.action_space.sample()
      n_state, reward, done, info = env.step(action)
      score+=reward
  if info['opp_score'] < score:
    wins += 1

  print('Episode:{} Score:{} Info:{}'.format(episode, score, info))

print('Win Rate:',wins/100)


Episode:1 Score:640 Info:{'opp_score': 360}
Episode:2 Score:413 Info:{'opp_score': 478}
Episode:3 Score:109 Info:{'opp_score': 609}
Episode:4 Score:405 Info:{'opp_score': 465}
Episode:5 Score:156 Info:{'opp_score': 576}
Episode:6 Score:635 Info:{'opp_score': 390}
Episode:7 Score:828 Info:{'opp_score': 258}
Episode:8 Score:429 Info:{'opp_score': 424}
Episode:9 Score:623 Info:{'opp_score': 383}
Episode:10 Score:637 Info:{'opp_score': 367}
Episode:11 Score:138 Info:{'opp_score': 593}
Episode:12 Score:622 Info:{'opp_score': 362}
Episode:13 Score:87 Info:{'opp_score': 662}
Episode:14 Score:120 Info:{'opp_score': 535}
Episode:15 Score:806 Info:{'opp_score': 291}
Episode:16 Score:462 Info:{'opp_score': 457}
Episode:17 Score:629 Info:{'opp_score': 349}
Episode:18 Score:132 Info:{'opp_score': 592}
Episode:19 Score:156 Info:{'opp_score': 581}
Episode:20 Score:826 Info:{'opp_score': 261}
Episode:21 Score:286 Info:{'opp_score': 531}
Episode:22 Score:132 Info:{'opp_score': 602}
Episode:23 Score:592

# Deep Learning Model

In [18]:
env.state

array([0, 0, 0, 1, 1, 1, 0, 0, 1, 1])

In [19]:
states = env.observation_space.shape[0]
actions = env.action_space.n

In [20]:
states

10

In [21]:
actions

2

In [22]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(1,states)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(actions, activation='softmax'))
    model.add(Flatten())
    return model

In [23]:
# del model

In [24]:
model = build_model(states, actions)

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 128)            1408      
                                                                 
 dense_1 (Dense)             (None, 1, 128)            16512     
                                                                 
 dense_2 (Dense)             (None, 1, 128)            16512     
                                                                 
 dense_3 (Dense)             (None, 1, 2)              258       
                                                                 
 flatten (Flatten)           (None, 2)                 0         
                                                                 
Total params: 34,690
Trainable params: 34,690
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.input_shape

(None, 1, 10)

In [27]:
env.state.shape

(10,)

In [28]:
env.state

array([0, 0, 0, 1, 1, 1, 0, 0, 1, 1])

In [29]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3),metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [31]:
scores = dqn.test(env, nb_episodes=100, visualize=False,verbose=0)

In [32]:
import numpy as np

wins=0

for episode in range(100):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = dqn.forward(state)
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        state = next_state

        if done:
          if total_reward > info['opp_score']:
              wins +=1


print(f'Win Rate: {wins/100}')


Win Rate: 0.87


# Custom Model (Extra)

In [35]:
class ReplayBuffer:
    def __init__(self, buffer_size=int(1e5)):
        self.buffer = deque(maxlen=buffer_size)

    def store(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)


In [36]:
class MyDQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = ReplayBuffer()
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.state_size, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mae', optimizer=tf.keras.optimizers.legacy.Adam(lr=self.learning_rate))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def learn(self, batch_size):
        minibatch = self.memory.sample(batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [37]:
env = PrisonEnv()
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = MyDQNAgent(state_size, action_size)
episodes = 20

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    episode_reward=0
    for time in range(200):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        episode_reward += episode_reward
        next_state = np.reshape(next_state, [1, state_size])
        agent.memory.store(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(f"Episode: {e+1}/{episodes}, Score: {episode_reward}, Epsilon: {agent.epsilon:.2}")
            break
        if len(agent.memory) > 32:
            agent.learn(32)


c:\Users\Mohammad Doleh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Episode: 1/20, Score: 0, Epsilon: 0.43
Episode: 2/20, Score: 0, Epsilon: 0.16
Episode: 3/20, Score: 0, Epsilon: 0.059
Episode: 4/20, Score: 0, Epsilon: 0.022
Episode: 5/20, Score: 0, Epsilon: 0.01
Episode: 6/20, Score: 0, Epsilon: 0.01
Episode: 7/20, Score: 0, Epsilon: 0.01
Episode: 8/20, Score: 0, Epsilon: 0.01
Episode: 9/20, Score: 0, Epsilon: 0.01
Episode: 10/20, Score: 0, Epsilon: 0.01
Episode: 11/20, Score: 0, Epsilon: 0.01
Episode: 12/20, Score: 0, Epsilon: 0.01
Episode: 13/20, Score: 0, Epsilon: 0.01
Episode: 14/20, Score: 0, Epsilon: 0.01
Episode: 15/20, Score: 0, Epsilon: 0.01
Episode: 16/20, Score: 0, Epsilon: 0.01
Episode: 17/20, Score: 0, Epsilon: 0.01
Episode: 18/20, Score: 0, Epsilon: 0.01
Episode: 19/20, Score: 0, Epsilon: 0.01
Episode: 20/20, Score: 0, Epsilon: 0.01


In [38]:
agent.epsilon = 0
test_episodes = 100
wins=0

for episode in range(test_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    episode_reward = 0

    for t in range(500):
        action = agent.act(state)
        state, reward, done, info = env.step(action)
        state = np.reshape(state, [1, state_size])
        episode_reward += reward

        if done:
          if episode_reward > info['opp_score']:
            wins += 1
            break

    print(f"Test Episode {episode+1}/{test_episodes} Reward: {episode_reward} Opponent Score: {info['opp_score']}")

print(f"Win Rate: {wins/100}")


Test Episode 1/100 Reward: 212 Opponent Score: 207
Test Episode 2/100 Reward: 216 Opponent Score: 206
Test Episode 3/100 Reward: 509 Opponent Score: 509
Test Episode 4/100 Reward: 499 Opponent Score: 514
Test Episode 5/100 Reward: 212 Opponent Score: 207
Test Episode 6/100 Reward: 509 Opponent Score: 509
Test Episode 7/100 Reward: 212 Opponent Score: 207
Test Episode 8/100 Reward: 212 Opponent Score: 207
Test Episode 9/100 Reward: 509 Opponent Score: 509
Test Episode 10/100 Reward: 212 Opponent Score: 207
Test Episode 11/100 Reward: 212 Opponent Score: 207
Test Episode 12/100 Reward: 992 Opponent Score: 12
Test Episode 13/100 Reward: 509 Opponent Score: 509
Test Episode 14/100 Reward: 216 Opponent Score: 206
Test Episode 15/100 Reward: 509 Opponent Score: 509
Test Episode 16/100 Reward: 212 Opponent Score: 207
Test Episode 17/100 Reward: 583 Opponent Score: 118
Test Episode 18/100 Reward: 212 Opponent Score: 207
Test Episode 19/100 Reward: 992 Opponent Score: 12
Test Episode 20/100 Rew